### Extract Lyrics Embeddings

In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import scipy as sp

In [17]:
# import dataset
data = pd.read_csv('./spotify_data.csv', index_col=[0])
lyrics_data = pd.DataFrame(data['lyrics'])
lyrics_data['embedding'] = ''

In [15]:
lyrics_data

160          Aah Aah The games you played were never fun Yo...
445          The lights go out and I can't be saved Tides t...
763          'Cause you're a sky, 'cause you're a sky full ...
856          I will not make the same mistakes that you did...
1169         Heyy Heeey Heey Your lipstick stains On the fr...
                                   ...                        
16284        If you don't wanna see me Did a full 180, craz...
16771        Nah, nah, nah Cake by the ocean Oh, no See you...
17046        Come on, come on, turn the radio on It's Frida...
18402        Clock strikes upon the hour And the sun begins...
embedding                                                     
Name: lyrics, Length: 318, dtype: object

In [19]:
# define model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
lyrics_data['embedding'] = lyrics_data['lyrics'].apply(lambda e: np.array(model.encode(e)).squeeze())
lyrics_data

,lyrics,embedding
160,Aah Aah The games you played were never fun Yo...,"[-0.057876237, 0.014094057, -0.0038802002, -0...."
445,The lights go out and I can't be saved Tides t...,"[0.043968435, 0.0033812, 0.056344982, -0.01610..."
763,"'Cause you're a sky, 'cause you're a sky full ...","[-0.041232467, -0.021443496, 0.06120861, -0.02..."
856,I will not make the same mistakes that you did...,"[-0.09303109, -0.02796939, 0.033706456, -0.003..."
1169,Heyy Heeey Heey Your lipstick stains On the fr...,"[-0.055941347, -0.0352199, 0.040726993, -0.056..."
...,...,...
16203,"Yeah, you! Yeah, you! I used to wanna be Livin...","[-0.07893406, -0.07447552, 0.050695658, -0.006..."
16284,"If you don't wanna see me Did a full 180, craz...","[-0.04479932, -0.10902327, 0.07418224, 0.01864..."
16771,"Nah, nah, nah Cake by the ocean Oh, no See you...","[-0.09902718, -0.06675105, 0.02859425, -0.0443..."
17046,"Come on, come on, turn the radio on It's Frida...","[-0.08326284, 0.0061422116, -0.028874382, -0.0..."


In [21]:
import pickle
with open('./lyrics_embedding.pkl', 'wb') as f:
    pickle.dump(lyrics_data, f)